<a href="https://colab.research.google.com/github/LaurensSluyterman/Geneeskunde-loter/blob/main/Geneeskunde_loter_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geneeskunde verdeler

Voer de volgende stappen uit:

upload het voorkeuren.csv bestand (het moet zo heten!) naar de sample_data map. Belangrijk is dat de 1e rij Naam, Voorkeur1, Voorkeur2,Voorkeur3, Voorkeur4, Voorkeur5 is! Je kan de namen anonimiseren als je wilt. Een input bestand ziet er dus zo uit:

Naam,Voorkeur1,Voorkeur2,Voorkeur3,Voorkeur4,Voorkeur5

Stefan, 1, 4, 2, 9, 3

Dayenne, 4, 8, 1, 21, 23

etc.

Run nu de code (het driehoekje in het vakje hieronder). Er worden 3 verschillende verdelingen gemaakt. De code probeert lagere voorkeuren in toenemende mate te vermijden. Verdeling 2 zal dus over het algemeen minder mensen een lagere voorkeur geven (maar dus ook minder mensen hun eerste voorkeur om dat te bewerkstelligen).


Voor vragen kan je mailen naar Laurens.Sluyterman at hotmail.com

In [ ]:
#@title <-- klik hier { display-mode: "form" }
"""Author: Laurens Sluijterman"""
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from datetime import date

data_directory = 'sample_data'
data_file_path = data_directory + "/voorkeuren.csv"
data = pd.read_csv(data_file_path)
data = data.sample(frac=1) # Reorder the data to avoid any possible advantage
number_of_students = len(data)
names = data.to_numpy()[:, 0]
preferences = data.drop(columns="Naam")
number_of_positions = int(input("Hoeveel verschillende plekken zijn er?"))
number_of_preferences = np.shape(preferences)[1]

# Waarschuwingen als er iets niet klopt aan de input
for i, preference in enumerate(preferences.to_numpy()):
    if len(np.unique(preference, return_counts=True)[1]) != len(preference):
        raise ImportWarning(f"{names[i]} heeft een dubbele voorkeur opgegeven")
    if preference.max() > number_of_positions:
        raise ImportWarning(f"{names[i]} heeft een niet bestaande voorkeuren opgegeven")
    if preference.min() < 1:
        raise ImportWarning(f"{names[i]} heeft een niet bestaande voorkeuren opgegeven")

if number_of_students > number_of_positions:
    raise ImportWarning(f"Er zijn meer studenten dan plaatsen")


cost_arrays = np.zeros((3, number_of_preferences))
cost_arrays[0] = [i for i in range(number_of_preferences)] #Linear cost
cost_arrays[1] = [i**2 for i in range(number_of_preferences)] #quadratic cost
cost_arrays[2] = [10**(i) for i in range(number_of_preferences)] #exponential cost
for k in range(0, 3):
    standard_cost_array = cost_arrays[k]
    cost_factor_no_preference = 10 * standard_cost_array[-1] # Ook subjectief, bepaalt hoe erg je een plek buiten je voorkeur vindt.

    cost_matrix = np.ones((number_of_students, number_of_positions)) * cost_factor_no_preference
    for i in range(number_of_students):
        for j in range(number_of_preferences):
            j_th_preference = preferences.to_numpy()[i][j] - 1  # Correct for counting at 1 in preference file
            cost_matrix[i, j_th_preference] = standard_cost_array[j]

    # Deze functie zorgt voor de daadwerkelijke toewijzing.
    student_numbers, student_assignments = linear_sum_assignment(cost_matrix)
    student_assignments += 1  # convert to regular counting (python starts at 0)

    print(f'Verdeling compleet, opgeslagen in het plaatsingen_{k+1}.txt bestand')
    # Dit gedeelte schrijft de resultaten naar een .txt file
    with open(data_directory + f"/plaatsingen_{k+1}.txt", "w") as myfile:
        myfile.write("Plaatsing gemaakt op " + str(date.today()) + "\n")
        myfile.write(f"Penalty waarden: {standard_cost_array} \n")
        remaining_positions = [i for i in range(1, number_of_positions + 1)]
        preference_count = np.zeros((number_of_preferences + 1))
        for i, preference in enumerate(preferences.to_numpy()):
            assignment = student_assignments[i]
            remaining_positions.remove(assignment)
            if assignment in preference:
                preference_number = np.where(preference == assignment)[0][0] + 1
                preference_count[preference_number-1] += 1
                myfile.write(f"{names[i]} --- {assignment} ({preference_number}e voorkeur)\n")
            else:
                myfile.write(f"{names[i]} --- {student_assignments[i]} (buiten voorkeuren) \n")
                preference_count[-1] += 1
        myfile.write(f"Overgebleven plaatsen: {remaining_positions} \n")
        print(f'Verdeling {k+1}:')
        for l, count in enumerate(preference_count):
            if l+1 <= number_of_preferences:
                myfile.write(f"{l+1}e voorkeur --- {count} x \n")
                print(f"{l+1}e voorkeur --- {count} x \n")
            else:
                myfile.write(f"Buiten voorkeuren --- {count} x \n")
                print(f"Buiten voorkeuren --- {count} x \n")


Hoeveel verschillende plekken zijn er?50
Verdeling compleet, opgeslagen in het plaatsingen_-1.txt bestand
Verdeling 1:
1e voorkeur --- 15.0 x 

2e voorkeur --- 8.0 x 

3e voorkeur --- 1.0 x 

4e voorkeur --- 3.0 x 

5e voorkeur --- 2.0 x 

Buiten voorkeuren --- 0.0 x 

Verdeling compleet, opgeslagen in het plaatsingen_0.txt bestand
Verdeling 2:
1e voorkeur --- 13.0 x 

2e voorkeur --- 9.0 x 

3e voorkeur --- 4.0 x 

4e voorkeur --- 2.0 x 

5e voorkeur --- 1.0 x 

Buiten voorkeuren --- 0.0 x 

Verdeling compleet, opgeslagen in het plaatsingen_1.txt bestand
Verdeling 3:
1e voorkeur --- 11.0 x 

2e voorkeur --- 7.0 x 

3e voorkeur --- 5.0 x 

4e voorkeur --- 6.0 x 

5e voorkeur --- 0.0 x 

Buiten voorkeuren --- 0.0 x 

